In [1]:
%%capture
%pip install unsloth
%pip uninstall unsloth -y && pip install --upgrade --no-cache-dir --no-deps git+https://github.com/unslothai/unsloth.git

In [2]:
import os
from dotenv import load_dotenv
load_dotenv()

from huggingface_hub import login
login(os.getenv('HUGGINGFACE_TOKEN'))

In [3]:
from unsloth import FastLanguageModel
import torch

max_seq_length = 2048
dtype = None
load_in_4bit = True

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "Llama1B_FineTuned",
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
)
FastLanguageModel.for_inference(model)

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!
==((====))==  Unsloth 2025.3.14: Fast Llama patching. Transformers: 4.49.0.
   \\   /|    Tesla T4. Num GPUs = 1. Max memory: 14.748 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.6.0+cu124. CUDA: 7.5. CUDA Toolkit: 12.4. Triton: 3.2.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.29.post3. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


Unsloth 2025.3.14 patched 16 layers with 16 QKV layers, 16 O layers and 16 MLP layers.


PeftModelForCausalLM(
  (base_model): LoraModel(
    (model): LlamaForCausalLM(
      (model): LlamaModel(
        (embed_tokens): Embedding(128256, 2048, padding_idx=128004)
        (layers): ModuleList(
          (0): LlamaDecoderLayer(
            (self_attn): LlamaAttention(
              (q_proj): lora.Linear4bit(
                (base_layer): Linear4bit(in_features=2048, out_features=2048, bias=False)
                (lora_dropout): ModuleDict(
                  (default): Identity()
                )
                (lora_A): ModuleDict(
                  (default): Linear(in_features=2048, out_features=16, bias=False)
                )
                (lora_B): ModuleDict(
                  (default): Linear(in_features=16, out_features=2048, bias=False)
                )
                (lora_embedding_A): ParameterDict()
                (lora_embedding_B): ParameterDict()
                (lora_magnitude_vector): ModuleDict()
              )
              (k_proj): lora.Linear

In [ ]:
from src.utils import generate_prompt

input = {"input": "You are a name and surname extractor. Your task is to identify and extract the names and surnames of characters from the given text.\n### Input:\nIn the quaint town of Eldridge, Sarah Quinn and Lucas Bennett stood at the park, deep in conversation. The autumn leaves crunched underfoot as they walked along the winding path. \n\n\"I can't believe we’re finally doing this,\" Lucas said, a nervous excitement in his voice.\n\nSarah smiled, brushing a loose strand of hair behind her ear. \"I know! After months of planning, it feels surreal.\"\n\nThey stopped by the old oak tree, its branches stretching wide like welcoming arms. “Have you thought about what you’ll say?” Sarah asked, her blue eyes sparkling with mischief.\n\n“Just that I appreciate everything he’s done for us,” Lucas replied, kicking a pebble out of their way.\n\nAs they watched the sun dip below the horizon, Jack Caldwell approached, his camera slung across his shoulder. “The moment’s finally here, huh?” he grinned, ready to capture the proposal.\n\nSarah squeezed Lucas’s hand, a flutter of anticipation in her stomach as they prepared for a new chapter in their lives. Tonight, under the stars, they would begin a story of love that had been years in the making.\n\n### Instructions:\n1. Extract characters' names and surnames clearly.\n2. Ensure that the JSON file contains a list of unique name-surname pairs (no duplicates).\n\n### Format Instructions:\n1. Do NOT include headers, comments, or any additional text—only generate the JSON file.\n2. Strictly adhere to the following JSON format:\nThe output should be formatted as a JSON instance that conforms to the JSON schema below.\n\nAs an example, for the schema {\"properties\": {\"foo\": {\"title\": \"Foo\", \"description\": \"a list of strings\", \"type\": \"array\", \"items\": {\"type\": \"string\"}}}, \"required\": [\"foo\"]}\nthe object {\"foo\": [\"bar\", \"baz\"]} is a well-formatted instance of the schema. The object {\"properties\": {\"foo\": [\"bar\", \"baz\"]}} is not well-formatted.\n\nHere is the output schema:\n```\n{\"$defs\": {\"FullName\": {\"properties\": {\"name\": {\"description\": \"Name of the character\", \"title\": \"Name\", \"type\": \"string\"}, \"surname\": {\"description\": \"Surname of the character\", \"title\": \"Surname\", \"type\": \"string\"}}, \"required\": [\"name\", \"surname\"], \"title\": \"FullName\", \"type\": \"object\"}}, \"properties\": {\"fullname_list\": {\"description\": \"List of full names for every character\", \"items\": {\"$ref\": \"#/$defs/FullName\"}, \"title\": \"Fullname List\", \"type\": \"array\"}}, \"required\": [\"fullname_list\"]}\n```"}
input = {"input": "You are a professional chef creating detailed recipes. Create a complete recipe with precise measurements and clear instructions.\n### Instructions:\n1. Provide a clear, descriptive title for the recipe.\n2. List all ingredients with exact quantities and units.\n3. Write step-by-step cooking instructions in a logical order.\n4. Include total cooking time in minutes.\n5. Specify the cuisine type.\n6. Include nutrition facts when possible (calories, protein, fat, carbs).\n\n### Format Instructions:\n1. Do NOT include headers, comments, or any additional text—only generate the JSON file.\n2. Strictly adhere to the following JSON format:\nThe output should be formatted as a JSON instance that conforms to the JSON schema below.\n\nAs an example, for the schema {\"properties\": {\"foo\": {\"title\": \"Foo\", \"description\": \"a list of strings\", \"type\": \"array\", \"items\": {\"type\": \"string\"}}}, \"required\": [\"foo\"]}\nthe object {\"foo\": [\"bar\", \"baz\"]} is a well-formatted instance of the schema. The object {\"properties\": {\"foo\": [\"bar\", \"baz\"]}} is not well-formatted.\n\nHere is the output schema:\n```\n{\"$defs\": {\"Ingredient\": {\"properties\": {\"name\": {\"title\": \"Name\", \"type\": \"string\"}, \"quantity\": {\"title\": \"Quantity\", \"type\": \"number\"}, \"unit\": {\"title\": \"Unit\", \"type\": \"string\"}}, \"required\": [\"name\", \"quantity\", \"unit\"], \"title\": \"Ingredient\", \"type\": \"object\"}, \"Instruction\": {\"properties\": {\"step_number\": {\"title\": \"Step Number\", \"type\": \"integer\"}, \"description\": {\"title\": \"Description\", \"type\": \"string\"}}, \"required\": [\"step_number\", \"description\"], \"title\": \"Instruction\", \"type\": \"object\"}, \"NutritionFacts\": {\"properties\": {\"calories\": {\"title\": \"Calories\", \"type\": \"number\"}, \"protein\": {\"title\": \"Protein\", \"type\": \"number\"}, \"fat\": {\"title\": \"Fat\", \"type\": \"number\"}, \"carbs\": {\"title\": \"Carbs\", \"type\": \"number\"}}, \"required\": [\"calories\", \"protein\", \"fat\", \"carbs\"], \"title\": \"NutritionFacts\", \"type\": \"object\"}}, \"properties\": {\"title\": {\"title\": \"Title\", \"type\": \"string\"}, \"cuisine\": {\"title\": \"Cuisine\", \"type\": \"string\"}, \"ingredients\": {\"items\": {\"$ref\": \"#/$defs/Ingredient\"}, \"title\": \"Ingredients\", \"type\": \"array\"}, \"instructions\": {\"items\": {\"$ref\": \"#/$defs/Instruction\"}, \"title\": \"Instructions\", \"type\": \"array\"}, \"total_time_minutes\": {\"title\": \"Total Time Minutes\", \"type\": \"integer\"}, \"nutrition\": {\"anyOf\": [{\"$ref\": \"#/$defs/NutritionFacts\"}, {\"type\": \"null\"}], \"default\": null}}, \"required\": [\"title\", \"cuisine\", \"ingredients\", \"instructions\", \"total_time_minutes\"]}\n```"}
input = {"input": "You are a visionary game designer tasked with creating an innovative and engaging game concept.\n### Instructions:\n1. Create a unique game concept that combines compelling gameplay mechanics with an engaging narrative.\n2. Design 2-5 memorable characters with distinct personalities, motivations, and story arcs.\n3. Craft a rich setting that enhances the game's atmosphere and influences gameplay.\n4. Develop an intriguing plot with meaningful conflicts and satisfying resolution.\n5. Consider the game's genre and how it affects the overall experience.\n6. Specify the target platform(s) and technical requirements.\n7. Ensure the game idea is feasible and marketable while maintaining creativity.\n\n### Format Instructions:\n1. Do NOT include headers, comments, or any additional text—only generate the JSON file.\n2. Strictly adhere to the following JSON format:\nThe output should be formatted as a JSON instance that conforms to the JSON schema below.\n\nAs an example, for the schema {\"properties\": {\"foo\": {\"title\": \"Foo\", \"description\": \"a list of strings\", \"type\": \"array\", \"items\": {\"type\": \"string\"}}}, \"required\": [\"foo\"]}\nthe object {\"foo\": [\"bar\", \"baz\"]} is a well-formatted instance of the schema. The object {\"properties\": {\"foo\": [\"bar\", \"baz\"]}} is not well-formatted.\n\nHere is the output schema:\n```\n{\"$defs\": {\"Character\": {\"properties\": {\"name\": {\"description\": \"Name of the character\", \"title\": \"Name\", \"type\": \"string\"}, \"age\": {\"description\": \"Age of the character\", \"title\": \"Age\", \"type\": \"integer\"}, \"gender\": {\"description\": \"Gender of the character\", \"title\": \"Gender\", \"type\": \"string\"}, \"role\": {\"description\": \"Role of the character\", \"title\": \"Role\", \"type\": \"string\"}, \"biography\": {\"description\": \"Biography of the character\", \"title\": \"Biography\", \"type\": \"string\"}}, \"required\": [\"name\", \"age\", \"gender\", \"role\", \"biography\"], \"title\": \"Character\", \"type\": \"object\"}, \"Conflict\": {\"properties\": {\"name\": {\"description\": \"Name of the conflict\", \"title\": \"Name\", \"type\": \"string\"}, \"description\": {\"description\": \"Description of the conflict\", \"title\": \"Description\", \"type\": \"string\"}}, \"required\": [\"name\", \"description\"], \"title\": \"Conflict\", \"type\": \"object\"}, \"Genre\": {\"enum\": [\"action\", \"comedy\", \"drama\", \"horror\", \"mystery\"], \"title\": \"Genre\", \"type\": \"string\"}, \"Plot\": {\"properties\": {\"title\": {\"description\": \"Title of the plot\", \"title\": \"Title\", \"type\": \"string\"}, \"description\": {\"description\": \"Description of the plot\", \"title\": \"Description\", \"type\": \"string\"}}, \"required\": [\"title\", \"description\"], \"title\": \"Plot\", \"type\": \"object\"}, \"Resolution\": {\"properties\": {\"name\": {\"description\": \"Name of the resolution\", \"title\": \"Name\", \"type\": \"string\"}, \"description\": {\"description\": \"Description of the resolution\", \"title\": \"Description\", \"type\": \"string\"}}, \"required\": [\"name\", \"description\"], \"title\": \"Resolution\", \"type\": \"object\"}, \"Setting\": {\"properties\": {\"name\": {\"description\": \"Name of the setting\", \"title\": \"Name\", \"type\": \"string\"}, \"description\": {\"description\": \"Description of the setting\", \"title\": \"Description\", \"type\": \"string\"}}, \"required\": [\"name\", \"description\"], \"title\": \"Setting\", \"type\": \"object\"}}, \"properties\": {\"genre\": {\"allOf\": [{\"$ref\": \"#/$defs/Genre\"}], \"description\": \"Genre of the game\"}, \"characters\": {\"description\": \"Characters of the game\", \"items\": {\"$ref\": \"#/$defs/Character\"}, \"title\": \"Characters\", \"type\": \"array\"}, \"setting\": {\"allOf\": [{\"$ref\": \"#/$defs/Setting\"}], \"description\": \"Setting of the game\"}, \"plot\": {\"allOf\": [{\"$ref\": \"#/$defs/Plot\"}], \"description\": \"Plot of the game\"}, \"conflict\": {\"allOf\": [{\"$ref\": \"#/$defs/Conflict\"}], \"description\": \"Conflict of the game\"}, \"resolution\": {\"allOf\": [{\"$ref\": \"#/$defs/Resolution\"}], \"description\": \"Resolution of the game\"}, \"description\": {\"description\": \"Description of the game\", \"title\": \"Description\", \"type\": \"string\"}, \"type\": {\"description\": \"Type of the game\", \"title\": \"Type\", \"type\": \"string\"}, \"platform\": {\"description\": \"Platform of the game\", \"title\": \"Platform\", \"type\": \"string\"}, \"release_date\": {\"description\": \"Release date of the game\", \"title\": \"Release Date\", \"type\": \"string\"}, \"rating\": {\"description\": \"Rating of the game\", \"title\": \"Rating\", \"type\": \"number\"}}, \"required\": [\"genre\", \"characters\", \"setting\", \"plot\", \"conflict\", \"resolution\", \"description\", \"type\", \"platform\", \"release_date\", \"rating\"]}\n```"}
input = generate_prompt(input, train=False)
# print(input)

input_ids = tokenizer(input, padding=True, return_tensors = "pt")["input_ids"].to("cuda")

from transformers import TextStreamer
text_streamer = TextStreamer(tokenizer, skip_prompt = True)
_ = model.generate(input_ids, streamer = text_streamer, max_new_tokens = 1024, pad_token_id = tokenizer.eos_token_id)

In [5]:
from src.langchain_llm_wrapper import LlamaLLM

llm = LlamaLLM(model=model, tokenizer=tokenizer)

In [6]:
input = "You are a professional chef creating detailed recipes. Create a complete recipe with precise measurements and clear instructions.\n### Instructions:\n1. Provide a clear, descriptive title for the recipe.\n2. List all ingredients with exact quantities and units.\n3. Write step-by-step cooking instructions in a logical order.\n4. Include total cooking time in minutes.\n5. Specify the cuisine type.\n6. Include nutrition facts when possible (calories, protein, fat, carbs).\n\n### Format Instructions:\n1. Do NOT include headers, comments, or any additional text—only generate the JSON file.\n2. Strictly adhere to the following JSON format:\nThe output should be formatted as a JSON instance that conforms to the JSON schema below.\n\nAs an example, for the schema {\"properties\": {\"foo\": {\"title\": \"Foo\", \"description\": \"a list of strings\", \"type\": \"array\", \"items\": {\"type\": \"string\"}}}, \"required\": [\"foo\"]}\nthe object {\"foo\": [\"bar\", \"baz\"]} is a well-formatted instance of the schema. The object {\"properties\": {\"foo\": [\"bar\", \"baz\"]}} is not well-formatted.\n\nHere is the output schema:\n```\n{\"$defs\": {\"Ingredient\": {\"properties\": {\"name\": {\"title\": \"Name\", \"type\": \"string\"}, \"quantity\": {\"title\": \"Quantity\", \"type\": \"number\"}, \"unit\": {\"title\": \"Unit\", \"type\": \"string\"}}, \"required\": [\"name\", \"quantity\", \"unit\"], \"title\": \"Ingredient\", \"type\": \"object\"}, \"Instruction\": {\"properties\": {\"step_number\": {\"title\": \"Step Number\", \"type\": \"integer\"}, \"description\": {\"title\": \"Description\", \"type\": \"string\"}}, \"required\": [\"step_number\", \"description\"], \"title\": \"Instruction\", \"type\": \"object\"}, \"NutritionFacts\": {\"properties\": {\"calories\": {\"title\": \"Calories\", \"type\": \"number\"}, \"protein\": {\"title\": \"Protein\", \"type\": \"number\"}, \"fat\": {\"title\": \"Fat\", \"type\": \"number\"}, \"carbs\": {\"title\": \"Carbs\", \"type\": \"number\"}}, \"required\": [\"calories\", \"protein\", \"fat\", \"carbs\"], \"title\": \"NutritionFacts\", \"type\": \"object\"}}, \"properties\": {\"title\": {\"title\": \"Title\", \"type\": \"string\"}, \"cuisine\": {\"title\": \"Cuisine\", \"type\": \"string\"}, \"ingredients\": {\"items\": {\"$ref\": \"#/$defs/Ingredient\"}, \"title\": \"Ingredients\", \"type\": \"array\"}, \"instructions\": {\"items\": {\"$ref\": \"#/$defs/Instruction\"}, \"title\": \"Instructions\", \"type\": \"array\"}, \"total_time_minutes\": {\"title\": \"Total Time Minutes\", \"type\": \"integer\"}, \"nutrition\": {\"anyOf\": [{\"$ref\": \"#/$defs/NutritionFacts\"}, {\"type\": \"null\"}], \"default\": null}}, \"required\": [\"title\", \"cuisine\", \"ingredients\", \"instructions\", \"total_time_minutes\"]}\n```"
res = llm(input)
print(res)

/tmp/ipykernel_28041/1857286031.py:2: LangChainDeprecationWarning: The method `BaseLLM.__call__` was deprecated in langchain-core 0.1.7 and will be removed in 1.0. Use :meth:`~invoke` instead.
  res = llm(input)



{
  "title": "Creamy Garlic Parmesan Chicken",
  "cuisine": "Italian",
  "ingredients": [
    {
      "name": "Chicken breasts",
      "quantity": 4,
      "unit": "pieces"
    },
    {
      "name": "Olive oil",
      "quantity": 2,
      "unit": "tablespoons"
    },
    {
      "name": "Garlic, minced",
      "quantity": 4,
      "unit": "cloves"
    },
    {
      "name": "Heavy cream",
      "quantity": 1,
      "unit": "cup"
    },
    {
      "name": "Parmesan cheese, grated",
      "quantity": 1,
      "unit": "cup"
    },
    {
      "name": "Salt",
      "quantity": 1,
      "unit": "teaspoon"
    },
    {
      "name": "Black pepper, freshly cracked",
      "quantity": 1,
      "unit": "teaspoon"
    },
    {
      "name": "Fresh parsley, chopped",
      "quantity": 2,
      "unit": "tablespoons"
    }
  ],
  "instructions": [
    {
      "step_number": 1,
      "description": "Heat the olive oil in a large skillet over medium heat."
    },
    {
      "step_number": 2,
    